In [35]:
import pandas as pd
%matplotlib inline

In [36]:
data_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df = pd.read_csv(data_url)
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,7653,8145,8676,9216,9998,10582,11173,11831,12456,13036
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,949,964,969,981,989,998,1004,1029,1050,1076
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7377,7542,7728,7918,8113,8306,8503,8697,8857,8997
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,761,762,762,762,762,762,763,763,763,763
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,52,52,58,60,61,69,70,70,71,74


In [37]:
df = df.loc[df["Country/Region"] == "Egypt"] #change country name here

#remove unused columns
df.drop("Province/State", axis=1, inplace=True)
df.drop("Lat", axis=1, inplace=True)
df.drop("Long", axis=1, inplace=True)
#df.head()

In [38]:
df = df.groupby('Country/Region').sum()

#remove zero values
df = df.loc[:, (df != 0).any(axis=0)]

#remove the unused country column
df.reset_index(inplace=True)
df.drop('Country/Region', axis=1, inplace=True)
#df.head()

In [39]:
df = df.transpose()

#change to datetime
dformat = '%m/%d/%y'
df.index = pd.DatetimeIndex(pd.to_datetime(df.index, format=dformat))
#df.head()

In [40]:
df.reset_index(inplace=True)
df.rename(columns={'index':'ds', 0:'y'},inplace=True)
#df.head()

In [41]:
#prepare for Prophet
df.reset_index(inplace=True)
df.rename(columns={'index':'ds', 0:'y'},inplace=True)
#df.head()

In [42]:
from fbprophet import Prophet

#split data into training and test sets
train_to_index = round(len(df) * .95)
train_data = df.iloc[:train_to_index]
test_data = df.iloc[train_to_index:]

#print('training data length',len(train_data),'test data length',len(test_data))

In [43]:
model = Prophet(growth ='linear',
seasonality_mode='multiplicative',
changepoint_prior_scale=0.5,
weekly_seasonality=False,
yearly_seasonality=True,
daily_seasonality=True)
model.fit(train_data)

#generate a dataframe with future dates, frequency is set to daily
future = model.make_future_dataframe(periods=len(test_data),freq='D')

#predict values for future dates
forecast = model.predict(future)

ValueError: cannot assemble with duplicate keys

In [ ]:
model.plot(forecast);

In [ ]:
metric_df = forecast.set_index('ds')[['yhat']].join(df.set_index('ds').y).reset_index()
metric_df.tail()

In [ ]:
from sklearn.metrics import r2_score

r2_score(metric_df.y, metric_df.yhat)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(metric_df.y, metric_df.yhat)

In [ ]:
future = model.make_future_dataframe(periods=30, freq = 'D')

#predict values for future dates
forecast = model.predict(future)

#print only required columns ds, yhat, yhat_lower, yhat_upper
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(27)